# Module Installation

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!pip install Sastrawi

     |████████████████████████████████| 209 kB 6.7 MB/s 


In [ ]:
!pip install PySastrawi

     |████████████████████████████████| 210 kB 9.5 MB/s 


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 4.9 MB/s 
     |████████████████████████████████| 4.0 MB 15.8 MB/s 
     |████████████████████████████████| 1.2 MB 45.1 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 895 kB 52.2 MB/s 
     |████████████████████████████████| 596 kB 55.5 MB/s 
     |████████████████████████████████| 6.6 MB 33.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=6f69be352524e1e0f8a7b90669c2c682668b11fe59df43c9c250c83fb462d993
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 4.7 MB/s 


In [ ]:
!pip install sklearn-pycrfsuite

     |████████████████████████████████| 485 kB 14.3 MB/s 
  Created wheel for sklearn-pycrfsuite: filename=sklearn_pycrfsuite-0.4.0-py2.py3-none-any.whl size=11002 sha256=2d5902bd9d04e4b3caa52437d60239be60cf99436173d14be5506434a7cc6382
  Stored in directory: /root/.cache/pip/wheels/44/50/b1/295e502c6652f551dc99bfdb394a3fe2fcb47668e333391f38
  Created wheel for python-crfsuite-extension: filename=python_crfsuite_extension-0.9.7-cp37-cp37m-linux_x86_64.whl size=777574 sha256=9e7d06a6081297093092cf7b043801a20c1f1a429575847a736f3a65c1cb94eb
  Stored in directory: /root/.cache/pip/wheels/70/d7/bf/369f4d179407dbf3840132e4135fa61f0bf711f4496d0bf750
Successfully built sklearn-pycrfsuite python-crfsuite-extension


# Import Module

In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score
from skmultilearn.problem_transform import LabelPowerset, BinaryRelevance, ClassifierChain
from sklearn.tree import DecisionTreeClassifier
import pickle

# Import dataset

## Download dataset

In [ ]:
!gdown --id "1GC6bxpZMtHNAn-yVmsyaf6S_dEQckDEk" # Labeled data
!gdown --id "1tw86SarCzoTSX0jy-v3RV2mKxPsoH5zL" # human labeled data
!gdown --id "1Dfj2RCER-gYpiDU5YKOJDdJ0HGkDLYo0" # Unlabeled data
!gdown --id "1KnHY6IuV7yY2IFcPhGG7eq80g707dhaM" # Testing data
!gdown --id "1aHd2K73jcHOkQWhYoYjO_Get3osVslJw" # Testing Label

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1GC6bxpZMtHNAn-yVmsyaf6S_dEQckDEk
To: /content/machine_annotated_labeled_data_v1.xml
100% 5.93M/5.93M [00:00<00:00, 25.9MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1tw86SarCzoTSX0jy-v3RV2mKxPsoH5zL
To: /content/human_annotated_labeled_data.xml
100% 1.11M/1.11M [00:00<00:00, 8.03MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  c

## Open Dataset

In [ ]:
def xml_to_df(xml_file, labeled=True) :
  xtree = ET.parse(xml_file)
  xroot = xtree.getroot()
  items = []
  doc = []

  for node in xroot.iter('DOK'):
    id = node.find('ID').text
    labels = []
    if(labeled) :
      for c in node.iter('CLASS') :
        label_items = []
        for label in c.findall('LABEL') :
          labels.append(label.text)
    for p in node.iter('PERTANYAAN') :
      judul = p.find('JUDUL').text
      isi = p.find('ISI').text
    
    if(labeled) :
      items = [id, judul, isi, labels]
      column = ['ID','JUDUL', 'ISI', 'LABEL']
    else : 
      items = [id, judul, isi]
      column=['ID','JUDUL', 'ISI']

    doc.append(items)
  df = pd.DataFrame(doc,columns=column)
  return df

In [ ]:
labeled_data = xml_to_df('/content/machine_annotated_labeled_data_v1.xml')
human_labeled_data = xml_to_df('/content/human_annotated_labeled_data.xml')
unlabeled_data = xml_to_df('/content/unlabeled_data_v1.xml', False)
testing_data = xml_to_df('/content/testing_data_v1.xml',False)
testing_label = pd.read_csv('/content/gold_standard.csv', index_col=0)

In [ ]:
merged_testing = pd.merge(left=testing_data, right=testing_label, left_on='ID', right_on='ID')

## Ubah label menjadi bentuk binary 

In [ ]:
label = [
  "Kebidanan dan Kandungan", 
  "Penyakit Dalam",
  "Kesehatan Anak",
  "Kesehatan Kulit dan Kelamin",
  "Kesehatan Gizi",
  "Kesehatan Telinga, Hidung dan Tenggorokan (THT)",
  "Gigi",
  "Kesehatan Mata",
  "Bedah",
  "Kesehatan Jiwa",
  "Ortopedi (Tulang)",
  "Jantung dan Pembuluh Darah",
  "Urologi",
  "Saraf",
  "Pulmonologi (Paru)",
  "Umum"
]

### Labeled data

In [ ]:
for item in label:
  labeled_data[item] = None

In [ ]:
for idx, row in labeled_data.iterrows():
  labels = row['LABEL']
  for item in labels:
    row[item] = 1
  
  for item in label:
    if item not in labels:
      row[item] = 0

In [ ]:
temp = []
for index,row in labeled_data.iterrows() :
  temp.append(list(row[label]))

In [ ]:
labeled_data['LABEL'] = temp

In [ ]:
none = []
for i in range(len(temp)):
  if(temp[i]==[None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None]):
    none.append(i)

In [ ]:
labeled_data.drop(none, axis=0, inplace=True)
labeled_data = labeled_data.reset_index(drop=True)

In [ ]:
labeled_data = labeled_data.drop(columns=label)

### Testing Data

In [ ]:
temp = []
for index,row in merged_testing.iterrows() :
  temp.append(list(row[label]))

In [ ]:
merged_testing['LABEL'] = temp
merged_testing = merged_testing.drop(columns=label)

In [ ]:
merged_testing['LABEL'] = merged_testing["LABEL"].apply(np.asarray)

# Preprocessing

## Remove Null Values

In [ ]:
data_labeled = labeled_data.drop(labels=['ID'],axis=1)
data_unlabeled = unlabeled_data.drop(labels=['ID'],axis=1)
merged_testing = merged_testing.drop(labels=['ID'],axis=1)

In [ ]:
data_labeled.shape, data_unlabeled.shape

((11526, 3), (69983, 2))

In [ ]:
print("Missing values pada data labeled:", data_labeled.isnull().sum())
print("Missing values pada data unlabeled:", data_unlabeled.isnull().sum())
print("Missing values pada data testing:", merged_testing.isnull().sum())

Missing values pada data labeled: JUDUL    0
ISI      4
LABEL    0
dtype: int64
Missing values pada data unlabeled: JUDUL      9
ISI      581
dtype: int64
Missing values pada data testing: JUDUL    0
ISI      6
LABEL    0
dtype: int64


> Kita dapat mengabaikan missing value tersebut, atau dengan kata lain, kita dapat mengambil baris-baris yang memiliki column ISI tidak NULL.

In [ ]:
clean_labeled_data = data_labeled[data_labeled['ISI'].notna()]

In [ ]:
clean_unlabeled_data = data_unlabeled[data_unlabeled['ISI'].notna() & data_unlabeled['JUDUL'].notna()]

In [ ]:
clean_testing_data = merged_testing[merged_testing['ISI'].notna()]

In [ ]:
clean_labeled_data[['JUDUL', 'ISI']].isnull().sum()

JUDUL    0
ISI      0
dtype: int64

In [ ]:
clean_unlabeled_data[['JUDUL', 'ISI']].isnull().sum()

JUDUL    0
ISI      0
dtype: int64

In [ ]:
clean_testing_data[['JUDUL', 'ISI']].isnull().sum()

JUDUL    0
ISI      0
dtype: int64

In [ ]:
data_labeled = clean_labeled_data.copy()
data_unlabeled = clean_unlabeled_data.copy()
merged_testing = clean_testing_data.copy()

##Duplikasi(Redundansi)

In [ ]:
data_labeled.shape, data_unlabeled.shape

((11522, 3), (69393, 2))

In [ ]:
data_labeled_dup = data_labeled.copy()

# tmptup = tuple()
for index, row in data_labeled_dup.iterrows():
  row['LABEL'] = tuple(row['LABEL'])

data_unlabeled_dup = clean_unlabeled_data

In [ ]:
print("Duplikasi pada data labeled:", data_labeled_dup.duplicated().any())
print("Duplikasi pada data unlabeled:", data_unlabeled_dup.duplicated().any())

Duplikasi pada data labeled: True
Duplikasi pada data unlabeled: True


In [ ]:
# Handle duplikasi dengan drop_duplicates pada masing masing kolom

data_labeled_dropdup = data_labeled_dup.drop_duplicates()
data_unlabeled_dropdup = data_unlabeled_dup.drop_duplicates()

In [ ]:
data_labeled = data_labeled_dropdup.copy()
data_unlabeled = data_unlabeled_dropdup.copy()

In [ ]:
data_labeled.shape, data_unlabeled.shape

((11519, 3), (68811, 2))

## Normalisasi

In [ ]:
def normalize(text):
    res = text.lower().strip()
    res = re.sub("\d", "", res)
    res = re.sub("\s+", " ", res)
    res = re.sub("[^\w\s]", "", res)
    return res

## Remove HTML Tag

In [ ]:
def clean_html(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(text))
    return cleantext

## Remove Stopword

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
list_stopword = factory.get_stop_words()

def remove_stop_words(text):
  list_token = text.split(" ")
  res = []
  for token in list_token:
    if token not in list_stopword:
      res.append(token)

  return " ".join(res) 

## Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def stemming(text):
  list_token = text.split(" ")
  res = []
  for token in list_token:
    word = stemmer.stem(token)
    res.append(word)
  return " ".join(res) 

In [ ]:
list_data = [data_labeled['JUDUL'], data_labeled['ISI'], data_unlabeled['JUDUL'], data_unlabeled['ISI'], merged_testing['JUDUL'], merged_testing['ISI']]
list_corpus = []

for data in list_data:
  corpus = []
  for text in data:
    try:
      normalized_text = normalize(text)
      clean_text = remove_stop_words(normalized_text)
      stemmed_text = stemming(clean_text)
      clean_text = clean_html(stemmed_text)
      corpus.append(clean_text)
    except:
      corpus.append(text)
  list_corpus.append(corpus)

##  Data

### Labeled

In [ ]:
text = pd.DataFrame(columns=['JUDUL','ISI'])
text['JUDUL'] = list_corpus[0]
text['ISI'] = list_corpus[1]

In [ ]:
text= text['JUDUL'].str.cat(text['ISI'], sep =' ')

In [ ]:
text_label = []
for i in data_labeled['LABEL'] :
  text_label.append(np.array(i))
text_label = np.array(text_label)

### Unlabeled

In [ ]:
unlabeled = pd.DataFrame(columns=['JUDUL','ISI'])
unlabeled['JUDUL'] = list_corpus[2]
unlabeled['ISI'] = list_corpus[3]

In [ ]:
unlabeled= unlabeled['JUDUL'].str.cat(unlabeled['ISI'], sep =' ')

### Testing

In [ ]:
testing = pd.DataFrame(columns=['JUDUL','ISI'])
testing['JUDUL'] = list_corpus[4]
testing['ISI'] = list_corpus[5]

In [ ]:
testing = testing['JUDUL'].str.cat(testing['ISI'], sep =' ')

In [ ]:
testing_label = []
for i in merged_testing['LABEL'] :
  testing_label.append(np.array(i))
testing_label = np.array(testing_label)

# Model Build

## Model Builder

In [ ]:
from sklearn.metrics import hamming_loss, precision_score, recall_score

def build_model(model, mlb_estimator, xtrain, ytrain, xtest, ytest):
  classifier = mlb_estimator(model)
  classifier.fit(xtrain, ytrain)

  pred = classifier.predict(xtest)

  acc = accuracy_score(ytest, pred)
  f1 = f1_score(ytest, pred, average="macro")
  recall = recall_score(ytest, pred, average="macro")
  precision = precision_score(ytest, pred, average="macro")
  ham = hamming_loss(ytest, pred)
  res = {
      'accuracy':acc, 'hamming loss': ham, 'f1': f1,
      'precision': precision, 'recall': recall
    }
  print(res)

  return classifier

## Testing Model with Ground Truth

In [ ]:
def model_testing(model, mlb_estimator, xtrain, ytrain, embeddings):
  classifier = mlb_estimator(model)
  classifier.fit(xtrain, ytrain)

  test_data = embeddings.transform(testing)
  pred = classifier.predict(test_data)

  f1_macro = f1_score(testing_label, pred, average="macro")
  f1_micro = f1_score(testing_label, pred, average="micro")
  recall = recall_score(testing_label, pred, average="macro")
  precision = precision_score(testing_label, pred, average="macro")
  ham = hamming_loss(testing_label, pred)
  EMR = np.all(pred == testing_label, axis=1).mean()


  res = {
      'EMR':EMR, 'hamming loss': ham, 'F1-Macro': f1_macro, 'F1-Micro': f1_micro,
      'precision': precision, 'recall': recall,
    }
  print(res)

  return classifier

## Model Tester and Save CSV

In [ ]:
# To test model with testing data
def test_model_and_save_to_csv(model, test_vector, file_name):
  pred = model.predict(test_vector).toarray()
  res_testing = pd.DataFrame(columns=["ID"])
  res_testing['ID'] = testing_data["ID"]

  for index, lab in enumerate(label):
    res_testing[lab] = pred[:, index]

  res_testing.to_csv(file_name, index=False)

# Text Representation

## BERT

In [ ]:
from sentence_transformers import SentenceTransformer

# bert_model = SentenceTransformer('indobenchmark/indobert-base-p1')
bert_model = SentenceTransformer('indolem/indobert-base-uncased')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/indolem_indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
corpus_bert = bert_model.encode(text)

In [ ]:
train_corpus, test_corpus, train_label, test_label = train_test_split(
    corpus_bert, 
    text_label, 
    test_size=0.3,
    random_state=2022
)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import accuracy_score, f1_score

classifier_svc = LinearSVC()
multilabel_classifier = LabelPowerset(classifier_svc)
multilabel_classifier = multilabel_classifier.fit(train_corpus, train_label)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
pred_res = multilabel_classifier.predict(test_corpus)
acc = accuracy_score(test_label, pred_res)
f1 = f1_score(test_label, pred_res, average='macro')

print('Prediksi SVC')
print('Akurasi:', acc)
print('F1:', f1)

Prediksi SVC
Akurasi: 0.36226851851851855
F1: 0.521190203450005


## TF-IDF

### LabelPowerset

In [ ]:
from skmultilearn.problem_transform import LabelPowerset, ClassifierChain
from sklearn.tree import DecisionTreeClassifier

vectorizer = TfidfVectorizer()
train_vector = vectorizer.fit_transform(text).toarray()

from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(
    train_vector, 
    text_label, 
    test_size=0.3,
    random_state=2022
)

In [ ]:
print("Label Powerset:\n")
print("Linear SVC:")
model_tfidf_labelpowerset = build_model(LinearSVC(max_iter=2000), LabelPowerset, X_train, y_train, X_test, y_test)


print("Multinomial Naive Bayes:")
print(build_model(MultinomialNB(), LabelPowerset, X_train, y_train, X_test, y_test))

Label Powerset:

Linear SVC:
{'accuracy': 0.5257523148148148, 'hamming loss': 0.07002314814814815, 'f1': 0.6804401410256466, 'precision': 0.7453578690067773, 'recall': 0.6314175375957234}
Multinomial Naive Bayes:
{'accuracy': 0.40827546296296297, 'hamming loss': 0.10174334490740741, 'f1': 0.38254441328302874, 'precision': 0.6429653418432091, 'recall': 0.38478343264719267}
LabelPowerset(classifier=MultinomialNB(), require_dense=[True, True])


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_testing(LinearSVC(max_iter=2000), LabelPowerset, train_vector, text_label, vectorizer)

{'EMR': 0.5888394385484423, 'hamming loss': 0.05417665183156453, 'F1-Macro': 0.6981489615746075, 'F1-Micro': 0.7068087077350625, 'precision': 0.7113623778219043, 'recall': 0.693635304083672}


LabelPowerset(classifier=LinearSVC(max_iter=2000), require_dense=[True, True])

### ClassifierChain

In [ ]:
from skmultilearn.problem_transform import ClassifierChain

print("ClassifierChain:\n")
print("Linear SVC:")
model_svc_cc = build_model(LinearSVC(), ClassifierChain, X_train, y_train, X_test, y_test)

print("\nMultinomial NB:")
model_nb_cc = build_model(MultinomialNB(), ClassifierChain, X_train, y_train, X_test, y_test)


ClassifierChain:

Linear SVC:
{'accuracy': 0.4516210103040965, 'hamming loss': 0.0647304599145514, 'f1': 0.6893183604006587, 'precision': 0.7881444415672094, 'recall': 0.6179423509334859}

Multinomial NB:
{'accuracy': 0.02136215129429505, 'hamming loss': 0.11153870319175672, 'f1': 0.049552881347308994, 'precision': 0.6530713622594778, 'recall': 0.0276796719198553}


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_testing(LinearSVC(), ClassifierChain, train_vector, text_label, vectorizer)

{'EMR': 0.5231085244779186, 'hamming loss': 0.05285005135227662, 'F1-Macro': 0.7053480705199795, 'F1-Micro': 0.7116507121176745, 'precision': 0.7366386809185926, 'recall': 0.6840261657982138}


ClassifierChain(classifier=LinearSVC(), require_dense=[True, True])

### OneVsRestClassifier

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

print("OneVsRestClassifier:\n")
print("Linear SVC:")
print(build_model(LinearSVC(), OneVsRestClassifier, X_train, y_train, X_test, y_test))

OneVsRestClassifier:

Linear SVC:
{'accuracy': 0.43721064814814814, 'hamming loss': 0.06481481481481481, 'f1': 0.6881233959743719, 'precision': 0.8077136760373422, 'recall': 0.6065038877516131}
OneVsRestClassifier(estimator=LinearSVC())


### MultiOutputClassifier

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
print("MultiOutputClassifier:\n")
print("Linear SVC:")
print(build_model(LinearSVC(), MultiOutputClassifier, X_train, y_train, X_test, y_test))

print("\nMultinomial NB:")
print(build_model(MultinomialNB(), MultiOutputClassifier, X_train, y_train, X_test, y_test))

MultiOutputClassifier:

Linear SVC:
{'accuracy': 0.43721064814814814, 'hamming loss': 0.06481481481481481, 'f1': 0.6881233959743719, 'precision': 0.8077136760373422, 'recall': 0.6065038877516131}
MultiOutputClassifier(estimator=LinearSVC())

Multinomial NB:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.016493055555555556, 'hamming loss': 0.11610243055555555, 'f1': 0.04238486796188475, 'precision': 0.540189974266434, 'recall': 0.023036290480614664}
MultiOutputClassifier(estimator=MultinomialNB())


In [ ]:
model_testing(LinearSVC(), MultiOutputClassifier, train_vector, text_label, vectorizer)

{'EMR': 0.500171174255392, 'hamming loss': 0.05244351249572064, 'F1-Macro': 0.7002561738643247, 'F1-Micro': 0.7108646926978885, 'precision': 0.7492675579158925, 'recall': 0.6659622832084917}


MultiOutputClassifier(estimator=LinearSVC())

### Binary Relevance

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
print("BinaryRelevance:\n")
print("Linear SVC:")
br_model = build_model(LinearSVC(), BinaryRelevance, X_train, y_train, X_test, y_test)

BinaryRelevance:

Linear SVC:
{'accuracy': 0.42472983161598393, 'hamming loss': 0.0643848957024378, 'f1': 0.6835564099731543, 'precision': 0.7976972122494281, 'recall': 0.6048585362946128}


In [ ]:
model_testing(LinearSVC(), BinaryRelevance, train_vector, text_label, vectorizer)

{'EMR': 0.500171174255392, 'hamming loss': 0.05244351249572064, 'F1-Macro': 0.7002561738643247, 'F1-Micro': 0.7108646926978885, 'precision': 0.7492675579158925, 'recall': 0.6659622832084917}


BinaryRelevance(classifier=LinearSVC(), require_dense=[True, True])

### MLKNN

In [ ]:
# !pip uninstall scikit-learn -y
# !pip install scikit-learn==0.24.1

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
     |████████████████████████████████| 22.3 MB 38.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.1 which is incompatible.


In [ ]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV

parameters = {'k': range(1,16)}
score = 'f1_macro'

clf = GridSearchCV(MLkNN(), parameters, scoring=score)
clf.fit(X_train, y_train)

print(clf.best_params_, clf.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result 

{'k': 14} 0.6844981452713705


In [ ]:
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV

classifier = MLkNN(k=14)

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=14 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [ ]:
acc = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average="macro")
recall = recall_score(y_test, predictions, average="macro")
precision = precision_score(y_test, predictions, average="macro")
ham = hamming_loss(y_test, predictions)
res = {
    'accuracy':acc, 'hamming loss': ham, 'f1': f1,
    'precision': precision, 'recall': recall
  }
print(res)

{'accuracy': 0.44994212962962965, 'hamming loss': 0.06935402199074074, 'f1': 0.6799616460797173, 'precision': 0.7618042557184039, 'recall': 0.6225111864181876}


### Testing best model with testing data

In [ ]:
testing_data_vector = vectorizer.transform(testing)
testing_data_vector

<2921x23004 sparse matrix of type '<class 'numpy.float64'>'
	with 117209 stored elements in Compressed Sparse Row format>

In [ ]:
pred = model.predict(testing_data_vector).A
res_testing = pd.DataFrame(columns=["ID"])
res_testing['ID'] = testing_data["ID"]

for index, lab in enumerate(label):
  res_testing[lab] = pred[:, index]

res_testing.to_csv("cc_svc_tfidf.csv", index=False)

In [ ]:
test_model_and_save_to_csv(model_tfidf_labelpowerset, testing_data_vector.toarray(), 'tfidf_labelpowerset.csv')

## Bag of Word

In [ ]:
from skmultilearn.problem_transform import LabelPowerset, ClassifierChain
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

bowdf_vectorizer = CountVectorizer()

bow_train_vector_ = bowdf_vectorizer.fit_transform(text).toarray()

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(
    bow_train_vector_, 
    text_label, 
    test_size=0.3,
    random_state=2022
)

In [ ]:
print("Label Powerset for BoW:\n")
print("Linear SVC for BoW:")
print(build_model(LinearSVC(), LabelPowerset, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

print("\nDecision Tree for BoW:")
print(build_model(DecisionTreeClassifier(), LabelPowerset, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

Label Powerset for BoW:

Linear SVC for BoW:


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


{'accuracy': 0.4592013888888889, 'hamming loss': 0.08521412037037036, 'f1': 0.6258462920489611, 'precision': 0.6543048700406933, 'recall': 0.6012424727046255}
LabelPowerset(classifier=LinearSVC(), require_dense=[True, True])

Decision Tree for BoW:
{'accuracy': 0.3706597222222222, 'hamming loss': 0.10545066550925926, 'f1': 0.546996161586108, 'precision': 0.5562330316723572, 'recall': 0.5388416250821404}
LabelPowerset(classifier=DecisionTreeClassifier(), require_dense=[True, True])


In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import MultinomialNB

print("Classifier Chain untuk BoW:\n")
print("Linear SVC untuk BoW:")
print(build_model(LinearSVC(), ClassifierChain, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

print("\nMultinomial NB untuk BoW:")
print(build_model(MultinomialNB(), ClassifierChain, X_train_bow, y_train_bow, X_test_bow, y_test_bow))


Classifier Chain untuk BoW:

Linear SVC untuk BoW:


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

{'accuracy': 0.36747685185185186, 'hamming loss': 0.08559389467592593, 'f1': 0.6349836847666921, 'precision': 0.6516712360951136, 'recall': 0.6204431588310706}
ClassifierChain(classifier=LinearSVC(), require_dense=[True, True])

Multinomial NB untuk BoW:
{'accuracy': 0.37702546296296297, 'hamming loss': 0.07781756365740741, 'f1': 0.6188974714581794, 'precision': 0.7148738843988522, 'recall': 0.567460220064931}
ClassifierChain(classifier=MultinomialNB(), require_dense=[True, True])


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB

print("OneVsRestClassifier untuk BoW:\n")
print("Linear SVC untuk BoW:")
print(build_model(LinearSVC(), OneVsRestClassifier, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

print("\nMultinomial NB untuk BoW:")
print(build_model(MultinomialNB(), OneVsRestClassifier, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

OneVsRestClassifier untuk BoW:

Linear SVC untuk BoW:


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

{'accuracy': 0.35300925925925924, 'hamming loss': 0.08396629050925926, 'f1': 0.6434660519137703, 'precision': 0.6645061788526939, 'recall': 0.6250585337484594}
OneVsRestClassifier(estimator=LinearSVC())

Multinomial NB untuk BoW:
{'accuracy': 0.3648726851851852, 'hamming loss': 0.08038556134259259, 'f1': 0.6001158965293234, 'precision': 0.681252285405009, 'recall': 0.5545710373342311}
OneVsRestClassifier(estimator=MultinomialNB())


In [ ]:
from sklearn.multioutput import MultiOutputClassifier

print("MultiOutputClassifier untuk BoW:\n")
print("Linear SVC untuk BoW:")
print(build_model(LinearSVC(), MultiOutputClassifier, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

print("\nMultinomial NB untuk BoW:")
print(build_model(MultinomialNB(), MultiOutputClassifier, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

MultiOutputClassifier untuk BoW:

Linear SVC untuk BoW:


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

{'accuracy': 0.3527199074074074, 'hamming loss': 0.08400245949074074, 'f1': 0.6433893382290388, 'precision': 0.6643652441372911, 'recall': 0.6250585337484594}
MultiOutputClassifier(estimator=LinearSVC())

Multinomial NB untuk BoW:
{'accuracy': 0.3648726851851852, 'hamming loss': 0.08038556134259259, 'f1': 0.6001158965293234, 'precision': 0.681252285405009, 'recall': 0.5545710373342311}
MultiOutputClassifier(estimator=MultinomialNB())


In [ ]:
from skmultilearn.problem_transform import BinaryRelevance

print("BinaryRelevance untuk BoW:\n")
print("Linear SVC untuk BoW:")
print(build_model(LinearSVC(), BinaryRelevance, X_train_bow, y_train_bow, X_test_bow, y_test_bow))

BinaryRelevance untuk BoW:

Linear SVC untuk BoW:


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

{'accuracy': 0.3527199074074074, 'hamming loss': 0.08400245949074074, 'f1': 0.643331542134653, 'precision': 0.6642861473807316, 'recall': 0.6249948881272782}
BinaryRelevance(classifier=LinearSVC(), require_dense=[True, True])


## Word2Vec

Model Word2Vec yang digunakan adalah dari id wiki yang sudah ditrain terlebih dahulu.

source: https://github.com/deryrahman/word2vec-bahasa-indonesia

In [ ]:
!gdown --id '1f6FelhvCNVN44aZd2gPLcbDmafAYfufV'
!gdown --id '14ytryALf8PYY9opAFe0WuUN7lYu0ZiJ_'
!gdown --id '1UG2qxFxBNVploiXGpUbuRGeZwWR0JtdW'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1f6FelhvCNVN44aZd2gPLcbDmafAYfufV
To: /content/idwiki_word2vec_300.model
100% 25.6M/25.6M [00:00<00:00, 83.8MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=14ytryALf8PYY9opAFe0WuUN7lYu0ZiJ_
To: /content/idwiki_word2vec_300.model.trainables.syn1neg.npy
100% 398M/398M [00:05<00:00, 78.5MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
 

In [ ]:
# Class untuk enkapsulasi beberapa fungsionalitas yang ada

from gensim import models
import re

class Word2VecModel:

    pretrained_file = "idwiki_word2vec_300.model"
    vector_size = 300
    model = models.word2vec.Word2Vec.load(pretrained_file)

    # Fungsi untuk mencari representasi vektor dari suatu kata  
    def get_word_embedding(self, word):
        try:
            return self.model[word]
        except: # Melakukan handling terhadap OOV
            return np.nan
    
    # Fungsi untuk mencari representasi vektor dari suatu kalimat
    def get_sentence_embedding(self, sent):
        lst_token = re.findall("[^\s]{3,}", sent.lower())
        rep = []
        for token in lst_token:
            try:
                token_rep = self.model[token]
                rep.append(token_rep)
            except: # Melakukan handling terhadap OOV
                continue
        return np.mean(rep, axis=0)
    
    # Fungsi untuk mengubah setiap teks pada suatu corpus menjadi representasi vektornya
    def transform(self, corpus):
        res = []
        for doc in corpus:
            tmp = self.get_sentence_embedding(doc)
            if type(tmp) == np.float64: # Melakukan handling terhadap OOV
                tmp = np.zeros(self.vector_size)
            res.append(tmp)
        return np.array(res)

In [ ]:
train_corpus_w2v, test_corpus_w2v, train_label_w2v, test_label_w2v = train_test_split(
    text, 
    text_label, 
    test_size=0.25,
    random_state=2022
)

In [ ]:
model_word2vec = Word2VecModel()

In [ ]:
train_vector_w2v = model_word2vec.transform(train_corpus_w2v)
test_vector_w2v = model_word2vec.transform(test_corpus_w2v)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score

classifier_svc = LinearSVC()
multilabel_classifier = LabelPowerset(classifier_svc)
multilabel_classifier = multilabel_classifier.fit(train_vector_w2v, train_label_w2v)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
pred_multi_w2v = multilabel_classifier.predict(test_vector_w2v)
acc = accuracy_score(test_label_w2v, pred_multi_w2v)
f1 = f1_score(test_label_w2v, pred_multi_w2v, average='macro')

print('Prediksi SVC')
print('Akurasi:', acc)
print('F1:', f1)

Prediksi SVC
Akurasi: 0.4270833333333333
F1: 0.5683360378655644
{'accuracy': 0.4270833333333333, 'hamming loss': 0.09741753472222223, 'f1': 0.5683360378655644, 'precision': 0.5912773945661286, 'recall': 0.5502038918100118}


# Semi Supervised Learning

In [ ]:
def semi_supervised(train, train_label, unlabeled, model, embeddings) :
  train_data = embeddings.fit_transform(train).toarray()
  model.fit(train_data, train_label)
  
  for i in range(100):
    j = i + 1
    test_data_raw = unlabeled[(i*10):(j*10)]
    test_data = embeddings.transform(test_data_raw).toarray()

    pred = model.predict(test_data)
    train_label = np.append(train_label, pred, axis=0)
    train = train.append(test_data_raw)
    train_data = embeddings.transform(train).toarray()
    model.fit(train_data, train_label)
  return model

In [ ]:
from sklearn.utils import shuffle
data, data_label = shuffle(text, text_label)

In [ ]:
unlabeled_data = shuffle(unlabeled)

## Ensemble

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier

from sklearn.ensemble import BaggingClassifier
bagging = AdaBoostClassifier(LinearSVC(), algorithm='SAMME')
modelll = build_model(bagging, LabelPowerset, X_train, y_train, X_test, y_test)

{'accuracy': 0.3402777777777778, 'hamming loss': 0.109375, 'f1': 0.44653333429516107, 'precision': 0.6676499856522975, 'recall': 0.39004658041299806}


In [ ]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(LinearSVC())
modell3 = build_model(bagging, LabelPowerset, X_train, y_train, X_test, y_test)

{'accuracy': 0.4777199074074074, 'hamming loss': 0.09195963541666667, 'f1': 0.5698760293584908, 'precision': 0.6105679379482718, 'recall': 0.5516314689918269}


In [ ]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(LinearSVC(max_iter=2000))
modell2 = build_model(bagging, MultiOutputClassifier, X_train, y_train, X_test, y_test)

{'accuracy': 0.4230324074074074, 'hamming loss': 0.06655092592592593, 'f1': 0.6662571714823278, 'precision': 0.8175273423737017, 'recall': 0.572252523719956}


In [ ]:
test_model_and_save_to_csv(modell2, testing_data_vector.toarray(), 'ensemble_bagging_csv_tfidf.csv')

## Self Training

#### Binary Relevance

In [ ]:
tfidf_vectorizer = TfidfVectorizer(token_pattern="[^\s]{3,}")
classifier_svc = LinearSVC()
classifier_svc = BinaryRelevance(classifier_svc)

In [ ]:
model_binary = semi_supervised(data, data_label, unlabeled_data, classifier_svc, tfidf_vectorizer)

In [ ]:
testing_data_vector = tfidf_vectorizer.transform(testing)
testing_data_vector

<2921x28755 sparse matrix of type '<class 'numpy.float64'>'
	with 110780 stored elements in Compressed Sparse Row format>

In [ ]:
test_model_and_save_to_csv(model_binary, testing_data_vector, 'SS_Binary.csv')

In [ ]:
# save the model to disk
filename = 'SS_Binary.sav'
pickle.dump(model_binary, open(filename, 'wb'))

### MultiOutputClassifier

In [ ]:
tfidf_vectorizer = TfidfVectorizer(token_pattern="[^\s]{3,}")
classifier_svc = LinearSVC()
classifier_svc = MultiOutputClassifier(classifier_svc)

In [ ]:
model_multioutput = semi_supervised(data, data_label, unlabeled_data, classifier_svc, tfidf_vectorizer)

In [ ]:
test_model_and_save_to_csv(model_multioutput, testing_data_vector, 'SS_Multioutput.csv')

In [ ]:
# save the model to disk
filename = 'SS_Multioutput.sav'
pickle.dump(model_multioutput, open(filename, 'wb'))

### LabelPowerset

In [ ]:
tfidf_vectorizer = TfidfVectorizer(token_pattern="[^\s]{3,}")
classifier_svc = LinearSVC()
classifier_svc = LabelPowerset(classifier_svc)

In [ ]:
train_data = tfidf_vectorizer.fit_transform(data).toarray()
classifier_svc.fit(train_data, data_label)

for i in range(100):
  j = i + 1
  test_data_raw = unlabeled_data[(i*10):(j*10)]
  test_data = tfidf_vectorizer.transform(test_data_raw).toarray()

  pred = classifier_svc.predict(test_data).A
  data_label = np.append(data_label, pred, axis=0)
  data = data.append(test_data_raw)
  train_data = tfidf_vectorizer.transform(data).toarray()
  classifier_svc.fit(train_data, data_label)

In [ ]:
model_label = semi_supervised(data, data_label, unlabeled_data, classifier_svc, tfidf_vectorizer)

In [ ]:
test_model_and_save_to_csv(model_label, testing_data_vector, 'SS_LabelPowerset.csv')

In [ ]:
# save the model to disk
filename = 'SS_LabelPowerset.sav'
pickle.dump(model_binary, open(filename, 'wb'))

# Analisa

In [ ]:
!gdown --id "13yS-XHCow4U1CreNEbqLyvTICiI24J3b"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13yS-XHCow4U1CreNEbqLyvTICiI24J3b
To: /content/SS_Binary.sav
100% 3.69M/3.69M [00:00<00:00, 189MB/s]


In [ ]:
model_multioutput = pickle.load(open('/content/SS_Binary.sav', 'rb'))

## Error Analysis

In [ ]:
def error_analysis(model, vectorizer):
  testing_text = merged_testing['JUDUL'] + " " + merged_testing["ISI"]
  testing_vector_err_analysis = vectorizer.transform(testing_text).toarray()
  predicted = model.predict(testing_vector_err_analysis).toarray()
  
  counter = {}
  ground_truth_label = merged_testing["LABEL"].to_numpy()
  for i in range(len(ground_truth_label)):
    for j in range(16):
      if ground_truth_label[i][j] == predicted[i][j]:
        counter[label[j]] = counter.get(label[j], 0) + 1


  for k,v in counter.items():
    print(k + ": ",v/len(testing_text))

### Supervised TFIDF LabelPowerset

In [ ]:
error_analysis(model_tfidf_labelpowerset, vectorizer)

Kebidanan dan Kandungan:  0.9311879493324204
Penyakit Dalam:  0.8801780212256076
Kesehatan Anak:  0.9568640876412188
Kesehatan Kulit dan Kelamin:  0.921602191030469
Kesehatan Gizi:  0.9640534063676823
Kesehatan Telinga, Hidung dan Tenggorokan (THT):  0.9565217391304348
Gigi:  0.983567271482369
Kesehatan Mata:  0.9887024991441288
Bedah:  0.9270797672030127
Kesehatan Jiwa:  0.9609722697706266
Ortopedi (Tulang):  0.9630263608353303
Jantung dan Pembuluh Darah:  0.9753509072235536
Urologi:  0.9486477233824033
Saraf:  0.9760356042451215
Pulmonologi (Paru):  0.9780896953098254
Umum:  0.8315645326942828


### Semi-supervised TFIDF Binary Relevance

In [ ]:
!gdown --id "1R0gGi3q3cphsq9HPVZTwFbYxJaGE1Qc3"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1R0gGi3q3cphsq9HPVZTwFbYxJaGE1Qc3
To: /content/SS_Binary.csv
100% 119k/119k [00:00<00:00, 1.68MB/s]


In [ ]:
ss_binary_df = pd.read_csv('SS_Binary.csv')
ss_binary_df.head(3)

,ID,Kebidanan dan Kandungan,Penyakit Dalam,Kesehatan Anak,Kesehatan Kulit dan Kelamin,Kesehatan Gizi,"Kesehatan Telinga, Hidung dan Tenggorokan (THT)",Gigi,Kesehatan Mata,Bedah,Kesehatan Jiwa,Ortopedi (Tulang),Jantung dan Pembuluh Darah,Urologi,Saraf,Pulmonologi (Paru),Umum
0,DS-45,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,DS-339,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AD-18,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [ ]:
merged_testing.head(3)

,JUDUL,ISI,LABEL
0,kenapa telinga berdengung tidak bisa hilang ?,"selamat sore dokter , sudah 7 hari telinga say...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,apabila anak anda lambat berbicara,"salam kenal team doktersehat , anak saya berum...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,sakit perut setiap habis makan dan minum,"selamat malam dok , kurang lebih 2 minggu ini ...","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]"


In [ ]:
predicted_ss_binary = ss_binary_df[label].to_numpy()
predicted_ss_binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
counter = {}
for i in range(len(predicted_ss_binary)):
  label_testing = merged_testing['LABEL'].to_numpy()
  for j in range(16):
    if predicted_ss_binary[i][j] == label_testing[i][j]:
      counter[label[j]] = counter.get(label[j], 0) + 1

In [ ]:
for k,v in counter.items():
  print(k + ": ",v/len(merged_testing))

Kebidanan dan Kandungan:  0.9318726463539884
Penyakit Dalam:  0.875042793563848
Kesehatan Anak:  0.9582334816843546
Kesehatan Kulit dan Kelamin:  0.9318726463539884
Kesehatan Gizi:  0.9698733310510099
Kesehatan Telinga, Hidung dan Tenggorokan (THT):  0.9582334816843546
Gigi:  0.9808284833960972
Kesehatan Mata:  0.9900718931872646
Bedah:  0.921602191030469
Kesehatan Jiwa:  0.9630263608353303
Ortopedi (Tulang):  0.9623416638137624
Jantung dan Pembuluh Darah:  0.9804861348853132
Urologi:  0.9507018144471071
Saraf:  0.9770626497774735
Pulmonologi (Paru):  0.9787743923313934
Umum:  0.8301951386511469
